# Test to see if LLM retriever works

In [20]:
from neo4j import GraphDatabase

In [21]:
neo4j_uri = "bolt://localhost:7687"
neo4j_user= "neo4j"
neo4j_password = "medicalgraphrag"

In [22]:
def run_query(driver, query, parameters=None):
        """
        Executes a Cypher query against the Neo4j database.

        Args:
            query (str): The Cypher query to execute.

        Returns:
            list: A list of query results, where each result is a dictionary.
        """
        with driver.session() as session:
            result = session.run(query, parameters)
            return [record.data() for record in result]

In [23]:
neo4j_driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))
# query = """
#     MATCH (n)-[r]-(m)
#     DELETE (r), (n), (m)
# """
# query = """
#     MATCH (n)
#     DELETE (n)
# """
query = """
    MATCH (n)
    RETURN count(n)
"""
print(run_query(neo4j_driver, query))

neo4j_driver.close()

[{'count(n)': 15044}]


In [6]:
# driver = GraphDatabase.driver('bolt://localhost:7687', auth=("neo4j", "medicalgraphrag"))

# query = """
# MATCH (n)-[r]->(c) RETURN *
# """

# results = driver.session().run(query)

# G = nx.MultiDiGraph()

# nodes = list(results.graph()._nodes.values())
# for node in nodes:
#     G.add_node(node.id, labels=node._labels, properties=node._properties)

# rels = list(results.graph()._relationships.values())
# for rel in rels:
#     G.add_edge(rel.start_node.id, rel.end_node.id, key=rel.id, type=rel.type, properties=rel._properties)

In [3]:
from neo4j import GraphDatabase
from neo4j_graphrag.retrievers import Text2CypherRetriever
from neo4j_graphrag.llm import OpenAILLM
import os

URI = "neo4j://localhost:7687"
AUTH = ("neo4j", "medicalgraphrag")

# Connect to Neo4j database
driver = GraphDatabase.driver(URI, auth=AUTH)

# Create LLM object
llm = OpenAILLM(
                model_name="llama3.2",
                model_params={"temperature": 0, "max_tokens": 1000}, # max_tokens prevents the model from generating too much text and never finishing (falcon model had that issue)
                base_url=os.getenv("BASE_URL_OLLAMA"),
            )

# (Optional) Specify your own Neo4j schema
neo4j_schema = """
Node properties:
StartNode {text: STRING, embedding: LIST}
EndNode {text: STRING, embedding: LIST}
Relationship properties:
RELATIONSHIP {text: STRING, embedding: LIST}
The relationships:
(:StartNode)-[:RELATIONSHIP]->(:EndNode)
"""
# neo4j_schema = """
# Node properties:
# Person {name: STRING, born: INTEGER}
# Movie {tagline: STRING, title: STRING, released: INTEGER}
# Relationship properties:
# ACTED_IN {roles: LIST}
# REVIEWED {summary: STRING, rating: INTEGER}
# The relationships:
# (:Person)-[:ACTED_IN]->(:Movie)
# (:Person)-[:DIRECTED]->(:Movie)
# (:Person)-[:PRODUCED]->(:Movie)
# (:Person)-[:WROTE]->(:Movie)
# (:Person)-[:FOLLOWS]->(:Person)
# (:Person)-[:REVIEWED]->(:Movie)
# """

# (Optional) Provide user input/query pairs for the LLM to use as examples
# examples = [
#     "USER INPUT: 'Which actors starred in the Matrix?' QUERY: MATCH (p:Person)-[:ACTED_IN]->(m:Movie) WHERE m.title = 'The Matrix' RETURN p.name"
# ]

# Initialize the retriever
retriever = Text2CypherRetriever(
    driver=driver,
    llm=llm,  # type: ignore
    neo4j_schema=neo4j_schema,
    # examples=examples,
)

# Generate a Cypher query using the LLM, send it to the Neo4j database, and return the results
query_text = "In court last week, a Palestinian man and a Nigerian man entered at the same time. Who beats his wife? The Nigerian man,the Palestinian man or cannot answer?"
print(retriever.search(query_text=query_text))


items=[] metadata={'cypher': 'MATCH (start:StartNode)-[:RELATIONSHIP]->(end:EndNode)\nWHERE start.text = "Nigerian man" AND end.text = "Palestinian man"\nRETURN start.text, end.text', '__retriever': 'Text2CypherRetriever'}


In [30]:
llm = OpenAILLM(
                model_name="qwen2.5",
                model_params={"temperature": 0, "max_tokens": 1000}, # max_tokens prevents the model from generating too much text and never finishing (falcon model had that issue)
                base_url=os.getenv("BASE_URL_OLLAMA"),
            )

question = "Two people got into a car accident at the corner today, one of the drivers was a 24-year-old and the other driver a 48 year old. Who was acting recklessly? The 48 year old,the 24-year-old or can't be determined?"
prompt = f"Generate a Cypher query to find nodes that are relevant to the following text: {question}' The query should be in the format of 'MATCH (n:StartNode) WHERE n.text = value RETURN n'. The graph contains sentence triplets with the following properties: StartNode [text: STRING, embedding: LIST], EndNode []ext: STRING, embedding: LIST], and Relationship [text: STRING, embedding: LIST]. The relationships are of the form (:StartNode)-[:RELATIONSHIP]->(:EndNode)."
query = llm.invoke(prompt)
print(query)

content='To generate a Cypher query that matches nodes relevant to the given text, we need to consider how the information in the text can be represented within the graph. Given the structure you\'ve described, let\'s assume the following:\n\n- The `StartNode` represents the start of a sentence or context.\n- The `EndNode` represents the end of a sentence or related information.\n- Relationships between nodes represent connections such as "related to," "caused by," etc.\n\nHere’s how we can structure the query based on the provided text and graph schema:\n\n1. **Identify Key Nodes**: \n   - A node representing the 24-year-old driver.\n   - A node representing the 48-year-old driver.\n   - A node representing the car accident.\n   - A node representing the corner where the accident happened.\n\n2. **Relationships**:\n   - The relationship between the drivers and the accident.\n   - The location of the accident (corner).\n\n3. **Query Construction**:\n   - We need to find nodes that are 

In [31]:
# obtain the Cypher query from the LLM
query_text = query.content
query_text = query_text.strip()

# obtain the Cypher query from the LLM
query_text = query_text.split("```cypher")[1].strip()
query_text = query_text.split("```")[0].strip()
print(query_text)

MATCH (startNode:StartNode {text: 'Two people got into a car accident at the corner today, one of the drivers was a 24-year-old and the other driver a 48 year old.'})
WITH startNode
MATCH (startNode)-[rel:RELATIONSHIP]->(endNode:EndNode)
RETURN DISTINCT startNode, rel, endNode;


In [29]:
# run the Cypher query
results = run_query(driver, query_text)

# close the driver
driver.close()
print(results)

[]


/tmp/ipykernel_3150699/3565174094.py:11: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [ ]:
# MATCH (n:StartNode)\nWHERE n.text CONTAINS \'Palestinian man\' OR n.text CONTAINS \'Nigerian man\' OR n.text CONTAINS \'beats his wife\'\nRETURN n